In [1]:
import numpy as np , pandas as pd , copy
from dimensions import dimensions
from keys_list import keys_list
import statistics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pretrained_dict=np.load("../examples/googlenet.npy", encoding='latin1').item()


In [3]:
deep_copy=copy.deepcopy(pretrained_dict) 

In [ ]:
# print(dimensions)

In [4]:
data_frame=pd.DataFrame(deep_copy)
pd.set_option('precision', 14)

In [ ]:
# print(data_frame['conv1_7x7_s2']['weights'].min())
# print(data_frame['conv1_7x7_s2']['biases'].min())

In [5]:
min_list_biases=[]
max_list_biases=[]

min_list_weights=[]
max_list_weights=[]

In [6]:
for i in data_frame.keys():
    min_list_biases.append(data_frame[i]['biases'].min())
    max_list_biases.append(data_frame[i]['biases'].max())
    
    min_list_weights.append(data_frame[i]['weights'].min())
    max_list_weights.append(data_frame[i]['weights'].max())

In [ ]:
# print(min_list_biases)
# print(max_list_biases)

# print(min_list_weights)
# print(max_list_weights)

In [7]:
min_biases  = min(min_list_biases)
max_biases  = max(max_list_biases)
min_weights = min(min_list_weights)
max_weights = max(max_list_weights)
len_weights = max(max_list_weights) - min(min_list_weights)
len_biases  = max(max_list_biases) - min(min_list_biases)
print("min_biases  : {} ".format(min_biases))
print("max_biases  :   {} ".format(max_biases))

print("min_weights :  {} ".format(min_weights))
print("max_weights :   {} ".format(max_weights))

print("len_weights :  {} ".format(len_weights))
print("len_biases :   {} ".format(len_biases))

min_biases  : -15.751750946044922 
max_biases  :   8.003083229064941 
min_weights :  -1.104335904121399 
max_weights :   1.6037755012512207 
len_weights :  2.70811128616333 
len_biases :   23.754833221435547 


In [ ]:
# print(type(data_frame))

In [ ]:
# len(data_frame["conv1_7x7_s2"]["weights"].flatten())

In [ ]:
# len(data_frame["conv1_7x7_s2"]["biases"].flatten())

In [8]:
weights=data_frame["conv1_7x7_s2"]["weights"].flatten()
biases=data_frame["conv1_7x7_s2"]["biases"].flatten()

In [9]:
for i in keys_list:
    if i == "conv1_7x7_s2":
        continue
    temp_weights=data_frame[i]["weights"].flatten() 
    weights=np.concatenate((weights,temp_weights))

In [10]:
for i in keys_list:
    if i == "conv1_7x7_s2":
        continue
    temp_biases=data_frame[i]["biases"].flatten() 
    biases=np.concatenate((biases,temp_biases))

In [11]:
print(len(weights))
print(len(biases))

6990272
8280


In [12]:
# print(dimensions)
sum_weights=0
sum_biases=0

In [13]:
for i in dimensions:
    partial_sum_weights=0
    partial_sum_biases=0
    if i == 'loss3_classifier':
        partial_sum_weights=dimensions[i][0][0]*dimensions[i][0][1]
    else:    
        partial_sum_weights=dimensions[i][0][0]*dimensions[i][0][1]*dimensions[i][0][2]*dimensions[i][0][3]
    
    partial_sum_biases=dimensions[i][1]
    sum_biases+=partial_sum_biases
    sum_weights+=partial_sum_weights

In [ ]:
# print(sum_weights)
# print(sum_biases)

In [ ]:
# print(min(biases))
# print(max(biases))

# print(min(weights))
# print(max(weights))

In [ ]:
# maximum=0
# counter=0
# for i in weights:
#     i=abs(i)
#     length=len(str(i))
#     if length == 14:
#         print (i,counter)
#     counter+=1    
#     if length > maximum :
#         maximum = length
# print(maximum)
# print(max(len(str(weights))))

In [14]:
print(len(weights))
print(len(np.unique(weights)))
print(len(np.unique(weights))/len(weights))
# weights.shape
weights_labels=np.zeros(shape=(len(weights),))
weights_labels.shape
radius=0.01
steps = (max_weights - min_weights)/radius
# print(steps)
steps= int(steps)+1
steps

6990272
6697665
0.9581408277102808


271

In [15]:
def find_in_list(list_input,item):
    indices=[i for i,x in enumerate(list_input) if x==item] 
    return indices

In [16]:
def Average(lst): 
    return sum(lst) / len(lst) 

In [17]:
def analyzer(input_set,minimum,maximum,radius):
    output_labels=np.zeros(shape=(len(input_set),))
    output_centers = np.zeros(shape=(len(input_set),))
    steps = (maximum - minimum)/radius
    steps = int(steps)+1
    center=minimum + radius
    for i in range(steps):
        lowerbound = center-radius #inclusive
        upperbound = center+radius #non inclusive
        index=0
        for j in input_set :
            if j >= lowerbound and j < upperbound :
                output_labels[index]=i
                output_centers[index]=center
            index += 1
        center += radius
    unique_labels = np.unique(output_labels)
    unique_labels_indexes=[]
    output_average = np.zeros(shape=(len(input_set),))
#     unique_labels_indexes=np.zeros(shape=(len(unique_labels),))
    unique_labels_freqs=[]
#     index=0
    for i in unique_labels:
        a=find_in_list(output_labels,i)
        unique_labels_indexes.append(a)
        unique_labels_freqs.append(len(a))
#         unique_labels_indexes[index]= find_in_list(input_set,i)
#         index+=1

    for i in unique_labels_indexes :
        temp_values=[]
        for j in i :
            temp_values.append(input_set[j])
#         mean=statistics.mean(temp_values)   
        mean = Average(temp_values)
        for j in i:
            output_average[j]=mean
        mean=0
            
    len_output_labels = len(output_labels)
    len_unique_output_labels = len(np.unique(output_labels))
    percentage_main = len_unique_output_labels / len_output_labels
    percentage_unused = len_unique_output_labels / steps

    return  output_labels , output_centers, output_average , steps ,percentage_main ,percentage_unused ,len_unique_output_labels,len_output_labels,unique_labels_indexes,unique_labels_freqs

In [ ]:
biases_labels,biases_centers,biases_average,steps,percentage_main,percentage_unused,len_unique_biases_labels,len_biases_labels,biases_unique_labels_indexes,biases_unique_labels_freqs= analyzer(biases,min_biases,max_biases,0.01)
# print(biases_labels)

In [20]:
def truncator(input_set,size):#.2f
    out=[0] * len(input_set)
    counter=0
    for i in input_set:
        s=float(format(i,size))
        out[counter]=s
        counter+=1
    return out

In [21]:
a=truncator(biases,".3f")
print(len(biases))
print(len(np.unique(biases)))
print(len(np.unique(a)))
print(1-float(len(np.unique(a)))/float(len(biases)))

8280
8278
2658
0.6789855072463769


In [22]:
v=truncator(weights,".3f")
print(len(weights))
print(len(np.unique(weights)))
print(len(np.unique(v)))
print(1-float(len(np.unique(v)))/float(len(weights)))

6990272
6697665
1362
0.9998051577964349


In [23]:
b=truncator(weights,".4f")
print(len(weights))
print(len(np.unique(weights)))
print(len(np.unique(b)))
print(1-float(len(np.unique(b)))/float(len(weights)))

6990272
6697665
8236
0.9988217911978247


In [24]:
c=truncator(weights,".5f")
print(len(weights))
print(len(np.unique(weights)))
print(len(np.unique(c)))
print(1-float(len(np.unique(c)))/float(len(weights)))

6990272
6697665
45996
0.9934199985351071


In [25]:
d=truncator(weights,".6f")
print(len(weights))
print(len(np.unique(weights)))
print(len(np.unique(d)))
print(1-float(len(np.unique(d)))/float(len(weights)))

6990272
6697665
262424
0.962458685441711


In [26]:
e=truncator(weights,".7f")
print(len(weights))
print(len(np.unique(weights)))
print(len(np.unique(e)))
print(1-float(len(np.unique(e)))/float(len(weights)))

6990272
6697665
1449949
0.7925761687098871


In [27]:
f=truncator(weights,".8f")
print(len(weights))
print(len(np.unique(weights)))
print(len(np.unique(f)))
print(1-float(len(np.unique(f)))/float(len(weights)))

6990272
6697665
4985037
0.28686079740530845


In [28]:
g=truncator(weights,".9f")
print(len(weights))
print(len(np.unique(weights)))
print(len(np.unique(g)))
print(1-float(len(np.unique(g)))/float(len(weights)))

6990272
6697665
6620079
0.0529583112073464


In [ ]:
print(len_unique_biases_labels)
print(len_biases_labels)
print(percentage_main)
print(percentage_unused)
print(steps)

In [ ]:
with open('biases_labels.txt', 'w') as f:
    for item in biases_labels:
        f.write("%s\n" % item)
with open('biases_centers.txt', 'w') as f2:
    for item in biases_centers:
        f2.write("%s\n" % item) 
with open('biases.txt', 'w') as f3:
    for item in biases:
        f3.write("%s\n" % item) 
with open('biases_averages.txt', 'w') as f4:
    for item in biases_average:
        f4.write("%s\n" % item)         

In [ ]:
nums=[i for i in range(len(biases_unique_labels_freqs))]
plt.xlabel("Labels")
plt.ylabel("Freq")
plt.title("Labels Freqs")
plt.plot(nums,biases_unique_labels_freqs,color="green",linewidth=0.8)

In [ ]:
weights_labels,weights_centers,weights_average,weights_steps,weights_percentage_main,weights_percentage_unused,len_unique_weights_labels,len_weights_labels,weights_unique_labels_indexes,weights_unique_labels_freqs= analyzer(weights,min_weights,max_weights,0.01)

In [ ]:
print(len_unique_weights_labels)
print(len_weights_labels)
print(weights_percentage_main)
print(weights_percentage_unused)
print(weights_steps)

In [ ]:
with open('weights_labels.txt', 'w') as f:
    for item in weights_labels:
        f.write("%s\n" % item)
with open('weights_centers.txt', 'w') as f2:
    for item in weights_centers:
        f2.write("%s\n" % item) 
with open('weights.txt', 'w') as f3:
    for item in weights:
        f3.write("%s\n" % item) 
with open('weights_averages.txt', 'w') as f4:
    for item in weights_average:
        f4.write("%s\n" % item)     

In [ ]:
nums=[i for i in range(len(weights_unique_labels_freqs))]
plt.xlabel("Labels")
plt.ylabel("Freq")
plt.title("Labels Freqs")
plt.plot(nums,weights_unique_labels_freqs,color="green",linewidth=0.8)